In [1]:
Pu = ("A","G")
Py = ("T","C")

def Comp(seq):
   comp=""
   for i in range(len(seq)):
       if(seq[i] in Pu):
           if(seq[i]==Pu[0]):
                comp+=Py[0]
           else:
                comp+=Py[1]
       elif(seq[i] in Py):
           if (seq[i] == Py[0]):
                comp+=Pu[0]
           else:
                comp+=Pu[1]
       else:
           comp+=""
   return comp
def Frames(seq):
    Frames=[]
    for i in range(3):
        seqt=seq[i:]
        Frames.insert(i,seqt[:(len(seqt)-(len(seqt)%3))])
    seq=Comp(seq)
    seq=seq[::-1]
    for i in range(3):
        seqt = seq[i:]
        Frames.insert(i+3, seqt[:(len(seqt) - (len(seqt) % 3))])
    return Frames
def Translation(seq):
    def Process(seq):
        table = {
            'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
            'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
            'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
            'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
            'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
            'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
            'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
            'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
            'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
            'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
            'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
            'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
            'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
            'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
            'TAC': 'Y', 'TAT': 'Y', 'TAA': '', 'TAG': '',
            'TGC': 'C', 'TGT': 'C', 'TGA': '*', 'TGG': 'W',
        }
        protein = ""
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            protein += table[codon] + " "
        return protein
    aa = {
        'A': 'Ala', 'C': 'Cys', 'D': 'Asp', 'E': 'Glu',
        'F': 'Phe', 'G': 'Gly', 'H': 'His', 'I': 'Ile',
        'K': 'Lys', 'L': 'Lue', 'M': 'Met', 'N': 'Asn',
        'P': 'Pro', 'Q': 'Gln', 'R': 'Arg', 'S': 'Ser',
        'T': 'Thr', 'V': 'Val', 'W': 'Trp', 'Y': 'Tyr'
    }
    F = Frames(seq)
    Protein = []
    tran = []

    for i in F:
        Protein.append(Process(i))
    for i in Protein:
        s = ""
        for j in range(len(i)):
            if (i[j] == '*'):
                s += '* '
            elif (i[j] != " "):
                s += aa[i[j]] + " "
        tran.append(s)
    return Protein,tran
def ORF(seq):
    def ORFs(seq):

        Start = "ATG"
        Stop = ["TAA", "TAG", "TGA"]
        bool = j = 0
        orf = []

        while (j < len(seq)):
            s1 = s2 = None
            bool = 0
            for i in range(j, len(seq), 3):
                codon = seq[i:i + 3]
                if (codon == Start and bool == 0):
                    bool = 1
                    s1 = i

                if (codon in Stop and bool == 1):
                    s2 = i + 3

                    break

            if (s1 != None):
                if (s2 == None):

                    s2 = len(seq) - (len(seq[s1:]) % 3)
                    orf.append(seq[s1:s2])
                    j = len(seq)
                else:
                    j = s1 + 3
                    orf.append(seq[s1:s2])
            else:
                orf.append("")
                j = len(seq)

        return orf
    F = Frames(seq)
    orf = {'Frame 1': ORFs(F[0]), 'Frame 2': ORFs(F[1]),
           'Frame 3': ORFs(F[2]), 'Frame 4': ORFs(F[3]),
           'Frame 5': ORFs(F[4]), 'Frame 6': ORFs(F[5]),
           }

    return orf
def findLongORF(seq):
    orf=ORF(seq)
    New=[]
    New.extend(orf["Frame 1"])
    New.extend(orf["Frame 2"])
    New.extend(orf["Frame 3"])
    New.extend(orf["Frame 4"])
    New.extend(orf["Frame 5"])
    New.extend(orf["Frame 6"])
    gt=New[0]
    for i in range(1,len(New)):
        if(len(gt)<len(New[i])):
            gt=New[i]

    for i in orf:
        if(gt in orf[i]):
            index="Number "+str(orf[i].index(gt)+1)+" Translational ORF in "+i
    return index,gt
def findRatio(seq):
    A=seq.count("A")
    T=seq.count("T")
    netP1=2*(A+T)
    G = seq.count("G")
    C = seq.count("C")
    netP2 = 2 * (G + C)
    ratio=str(netP1)+" : "+str(netP2)
    return ratio

seq="TTTAGGACTTGACAGTTTCGGTCCGGAATTCCCGGGATCTACT"             \
    "ACAAACAAATTACATTCAATATGCTTTTCTCAAGAGACTTTATT"            \
    "ATGCTTTTGGCATTCGCCGCCGTTGGCGCTGTCGCTGCACCTGTTG"          \
    "CAGAGCCTCAACCAGAAGCCAAAGCTGATGCTTCTCCAGCGGACTAT"         \
    "GGCGACTACGGTGACTATGGCGATTACGGAAACTATGGAACATACGAGAA"      \
    "ATACCCATCCTCTTCCTCCGTCACACCTACACCAACACCAACACCAACTGGAT"   \
    "ATGGTAGCTATGCCAGCTACGGTACCTACAAGAACAAGAAAGTTAGAG"        \
    "AAGCCGCCCCAGAGCCTGCTCCAGTAGCAAAGCGTGATGATTATGGAAACTAT"   \
    "GGAGACTATGGTGACTACGGAAACTACGGCAACTACGAGAACTACCCTTCCG"    \
    "CTACCTCTTCTGCTACCTCTTCTGCCACCCCTTCAAGCTCCTCGTCAAGCTCT"   \
    "TCATCAGCTACACCAACACCAACTCCAACCGGTTACGGTTCATACGCAAGCTACGG"\
    "AGCATACAAGAACAAGAAAGCACGTGCTCCAGAAGCCAACCCAGAAGACTATGGAA"\
    "ACTACGGTGACTACGGTGATTACGGCAACTACGAGAACTACACCCCTG"        \
    "CCTCCACCTCCTCCACATCCACATCCACCGTCACCCCAACTCCCACTCCCTAC"   \
    "CCAACGGCTACACAACTACGGCTCCTAAGTACTACGACACTACAGCGCGAAGC"   \
    "GATACGTTCAGCTTAGCGATGACTATTGACAGTTTATTGTATACACACTCTTAGAC"\
    "GATGAATCGCCTTTAATATATGATATAATGATCAATGTTGACTGCCTCGCACTGAT"\
    "TGGAGAATGGATAAAAG"

f=Frames(seq)
print("Frame 1")
print(f[0])
print("Frame 2")
print(f[1])
print("Frame 3")
print(f[2])
print("Frame 4")
print(f[3])
print("Frame 5")
print(f[4])
print("Frame 6")
print(f[5])
print()

Result = Translation(seq)
print(Result[0][0])
print(Result[1][0])
print()
print(Result[0][1])
print(Result[1][1])
print()
print(Result[0][2])
print(Result[1][2])
print()
print(Result[0][3])
print(Result[1][3])
print()
print(Result[0][4])
print(Result[1][4])
print()
print(Result[0][5])
print(Result[1][5])
print()
print("Frame 1")
print(ORF(seq)["Frame 1"])
print("Frame 2")
print(ORF(seq)["Frame 2"])
print("Frame 3")
print(ORF(seq)["Frame 3"])
print("Frame 4")
print(ORF(seq)["Frame 4"])
print("Frame 5")
print(ORF(seq)["Frame 5"])
print("Frame 6")
print(ORF(seq)["Frame 6"])
print()
print(findLongORF(seq))
print(findRatio(seq))

Frame 1
TTTAGGACTTGACAGTTTCGGTCCGGAATTCCCGGGATCTACTACAAACAAATTACATTCAATATGCTTTTCTCAAGAGACTTTATTATGCTTTTGGCATTCGCCGCCGTTGGCGCTGTCGCTGCACCTGTTGCAGAGCCTCAACCAGAAGCCAAAGCTGATGCTTCTCCAGCGGACTATGGCGACTACGGTGACTATGGCGATTACGGAAACTATGGAACATACGAGAAATACCCATCCTCTTCCTCCGTCACACCTACACCAACACCAACACCAACTGGATATGGTAGCTATGCCAGCTACGGTACCTACAAGAACAAGAAAGTTAGAGAAGCCGCCCCAGAGCCTGCTCCAGTAGCAAAGCGTGATGATTATGGAAACTATGGAGACTATGGTGACTACGGAAACTACGGCAACTACGAGAACTACCCTTCCGCTACCTCTTCTGCTACCTCTTCTGCCACCCCTTCAAGCTCCTCGTCAAGCTCTTCATCAGCTACACCAACACCAACTCCAACCGGTTACGGTTCATACGCAAGCTACGGAGCATACAAGAACAAGAAAGCACGTGCTCCAGAAGCCAACCCAGAAGACTATGGAAACTACGGTGACTACGGTGATTACGGCAACTACGAGAACTACACCCCTGCCTCCACCTCCTCCACATCCACATCCACCGTCACCCCAACTCCCACTCCCTACCCAACGGCTACACAACTACGGCTCCTAAGTACTACGACACTACAGCGCGAAGCGATACGTTCAGCTTAGCGATGACTATTGACAGTTTATTGTATACACACTCTTAGACGATGAATCGCCTTTAATATATGATATAATGATCAATGTTGACTGCCTCGCACTGATTGGAGAATGGATAAA
Frame 2
TTAGGACTTGACAGTTTCGGTCCGGAATTCCCGGGATCTACTACAAACAAATTACATTCAATATGCTTTTCTCAAGAGACTTTATTATGCTTTTGGCATTC